In [ ]:
!wget -P ../inaturalist_data/ https://storage.googleapis.com/wandb_datasets/nature_12K.zip

--2025-04-14 11:17:07--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.195.155, 142.250.207.91, 142.250.182.123, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.195.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G  10.8MB/s    in 5m 36s  

2025-04-14 11:22:43 (10.8 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [7]:
import zipfile

def print_zip_structure(zip_file_path):
    """
    Prints the directory structure of a ZIP file, including all directories and subdirectories.
    Prints only the first 5 files, but all directories.
    
    Args:
        zip_file_path (str): Path to the ZIP file.
    """
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
            file_count = 0  # Counter for files
            for item in zip_file.namelist():
                # Check if the item is a directory
                if item.endswith('/'):
                    file_count = 0  # Reset file count for each directory
                    print(f"Directory: {item}")
                else:
                    # Print only the first 5 files
                    if file_count < 5:
                        print(f"File: {item}")
                        file_count += 1
    except zipfile.BadZipFile:
        print("Error: The file is not a valid ZIP archive.")

# Example usage
print_zip_structure("../inaturalist_data/nature_12K.zip")

Directory: inaturalist_12K/
File: inaturalist_12K/.DS_Store
Directory: inaturalist_12K/train/
Directory: inaturalist_12K/train/Plantae/
File: inaturalist_12K/train/Plantae/1dfc3916ad58af6ce9af9fc8b36ceedd.jpg
File: inaturalist_12K/train/Plantae/e49eeff2b27ae74351fdf8ffa3791307.jpg
File: inaturalist_12K/train/Plantae/519ae1c558dff337bb4084934e31a4a8.jpg
File: inaturalist_12K/train/Plantae/2232663628cd9c456a6d01c90ba961c4.jpg
File: inaturalist_12K/train/Plantae/e26227424586b97ab3377a567ac4f006.jpg
Directory: inaturalist_12K/train/Aves/
File: inaturalist_12K/train/Aves/d55072249be7621868a3e62cae31ac29.jpg
File: inaturalist_12K/train/Aves/e53ee219fded8973f5295c2c628b3391.jpg
File: inaturalist_12K/train/Aves/2d0243d0dc0c6132b4d982c6617fed69.jpg
File: inaturalist_12K/train/Aves/4fe826e09bc9e1de1d5ced2e12c9e3b8.jpg
File: inaturalist_12K/train/Aves/b92c670548d19740c14a9add63a4277b.jpg
Directory: inaturalist_12K/train/Amphibia/
File: inaturalist_12K/train/Amphibia/3a7954b5e4efa5ef52aeba3d792adc

# Q1 - Solution

In [16]:
import torch
import torch.nn as nn

class MyCNN(nn.Module):
    """
    A modular CNN architecture:
      - 5 x (Conv -> Activation -> MaxPool)
      - 1 Dense (fully connected) layer of n neurons
      - 1 Output layer of 10 neurons

    Accepts images of size (3 x H x W).
    After 5 max-pool operations (each halves H and W),
    final feature map size is (m x (H/32) x (W/32)) if H and W are multiples of 32.
    """


    def __init__(self,
                 in_channels=3,
                 num_filters=16,     # m
                 kernel_size=3,      # k
                 activation_fn=nn.ReLU,
                 dense_neurons=128,  # n
                 image_height=224,   # default
                 image_width=224     # default
                 ):
        """
        :param in_channels:   Number of input channels (3 for RGB images)
        :param num_filters:   m = number of filters in each Conv layer
        :param kernel_size:   k = kernel size of each Conv filter (k x k)
        :param activation_fn: Pytorch activation class, e.g., nn.ReLU
        :param dense_neurons: n = number of neurons in the fully connected layer
        :param image_height:  The height of the input image (assumed multiple of 32)
        :param image_width:   The width of the input image (assumed multiple of 32)
        """
        super(MyCNN, self).__init__()

        # We assume 'same' padding, i.e., output of conv has same spatial size
        # Then each MaxPool(2x2) halves the H and W each time.
        padding = kernel_size // 2
        Act = activation_fn  # for readability

        #-------------------------
        # 1) Block 1
        #   Conv(in_channels->m), Act, MaxPool(2x2)
        #-------------------------
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=num_filters,
            kernel_size=kernel_size,
            stride=1,
            padding=padding
        )
        self.act1 = Act()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        #-------------------------
        # 2) Block 2
        #   Conv(m->m), Act, MaxPool(2x2)
        #-------------------------
        self.conv2 = nn.Conv2d(
            in_channels=num_filters,
            out_channels=num_filters,
            kernel_size=kernel_size,
            stride=1,
            padding=padding
        )
        self.act2 = Act()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        #-------------------------
        # 3) Block 3
        #   Conv(m->m), Act, MaxPool(2x2)
        #-------------------------
        self.conv3 = nn.Conv2d(
            in_channels=num_filters,
            out_channels=num_filters,
            kernel_size=kernel_size,
            stride=1,
            padding=padding
        )
        self.act3 = Act()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        #-------------------------
        # 4) Block 4
        #   Conv(m->m), Act, MaxPool(2x2)
        #-------------------------
        self.conv4 = nn.Conv2d(
            in_channels=num_filters,
            out_channels=num_filters,
            kernel_size=kernel_size,
            stride=1,
            padding=padding
        )
        self.act4 = Act()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        #-------------------------
        # 5) Block 5
        #   Conv(m->m), Act, MaxPool(2x2)
        #-------------------------
        self.conv5 = nn.Conv2d(
            in_channels=num_filters,
            out_channels=num_filters,
            kernel_size=kernel_size,
            stride=1,
            padding=padding
        )
        self.act5 = Act()
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)

        #--------------------------------------
        # Compute final feature map dimension.
        # Each pool halves H and W => H/32, W/32
        #--------------------------------------
        reduced_height = image_height // 32
        reduced_width  = image_width // 32
        self.flatten_dim = num_filters * reduced_height * reduced_width

        # Dense layer
        self.fc1 = nn.Linear(self.flatten_dim, dense_neurons)
        self.act_fc1 = Act()

        # Output layer: 10 neurons
        self.output = nn.Linear(dense_neurons, 10)

    def forward(self, x):
        # x: (batch_size, 3, H, W)

        # Block 1
        x = self.conv1(x)    # (batch_size, m, H, W)
        x = self.act1(x)     # (batch_size, m, H, W)
        x = self.pool1(x)    # (batch_size, m, H/2, W/2)

        # Block 2
        x = self.conv2(x)    # (batch_size, m, H/2, W/2)
        x = self.act2(x)     # (batch_size, m, H/2, W/2)
        x = self.pool2(x)    # (batch_size, m, H/4, W/4)

        # Block 3
        x = self.conv3(x)    # (batch_size, m, H/4, W/4)
        x = self.act3(x)     # (batch_size, m, H/4, W/4)
        x = self.pool3(x)    # (batch_size, m, H/8, W/8)

        # Block 4
        x = self.conv4(x)    # (batch_size, m, H/8, W/8)
        x = self.act4(x)     # (batch_size, m, H/8, W/8)
        x = self.pool4(x)    # (batch_size, m, H/16, W/16)

        # Block 5
        x = self.conv5(x)    # (batch_size, m, H/16, W/16)
        x = self.act5(x)     # (batch_size, m, H/16, W/16)
        x = self.pool5(x)    # (batch_size, m, H/32, W/32)

        # Flatten
        x = x.view(x.size(0), -1)  # (batch_size, m * (H/32) * (W/32))

        # Dense
        x = self.fc1(x)            # (batch_size, dense_neurons)
        x = self.act_fc1(x)        # (batch_size, dense_neurons)

        # Output: 10 classes
        x = self.output(x)         # (batch_size, 10)

        return x

In [ ]:
import os
import zipfile
from PIL import Image
import torch
import torchvision.transforms as T

def extract_data_if_needed(zip_path, extract_dir):
    """
    Extracts the zip file into 'extract_dir' if that folder does not exist.
    """
    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir, exist_ok=True)
        print(f"Extracting {zip_path} to {extract_dir} ...")

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(path=extract_dir)

        print("Extraction done.")
    else:
        print(f"Directory {extract_dir} already exists. Skipping extraction.")

def load_single_image(image_path, resize=True, resize_dim=(224, 224)):
    """
    Loads a single image with PIL, applies transforms, and returns the tensor.
    Also prints shape info for clarity.
    :param image_path: Path to a single .jpg file
    :return: A PyTorch tensor of shape (1, 3, 224, 224)
    """
    # Only convert to Tensor; no resizing to 224x224
    transform = T.Compose([
        T.ToTensor()  # # from [0..255] PIL image to [0..1] float tensor, shape => (C, H, W), range [0,1]
    ])

    # If resize is True, add resizing to the transform
    if resize:
        print("Resizing image to resize_dim[0] x resize_dim[1] ...")
        # Basic transform: resize to resize_dim[0] x resize_dim[1], then convert to tensor
        transform = T.Compose([
            T.Resize(resize_dim),  # (H, W)
            T.ToTensor()           # from [0..255] PIL image to [0..1] float tensor, shape (C,H,W)
        ])

    # Load the image
    pil_img = Image.open(image_path).convert('RGB')
    print(f"Original image size: {pil_img.size} (Width x Height)")

    # Apply transforms
    img_tensor = transform(pil_img)  # shape: (3, H, W)
    print(f"Transformed image shape: {img_tensor.shape} (C, H, W)")

    # Add a batch dimension => (1, 3, H, W) i.e. (batch_size, C, H, W)
    img_tensor = img_tensor.unsqueeze(0)
    print(f"Final input shape to the model: {img_tensor.shape} (Batch, C, H, W)\n")

    return img_tensor

def test_model_with_image(model, image_tensor):
    """
    Passes a single image tensor through the model and prints output shape.
    :param model: Instance of MyCNN
    :param image_tensor: shape (1, 3, H, W)
    """
    with torch.no_grad():
        output = model(image_tensor)
    print(f"Model output shape: {output.shape} (Batch, 10)")
    print("Raw output logits:", output)

def main():
    # Paths (adjust if necessary)
    DATA_ZIP_PATH = "../inaturalist_data/nature_12K.zip"
    EXTRACT_DIR   = "../inaturalist_data/nature_12K_extracted"
    
    # 1) Optional: Extract ZIP if needed
    extract_data_if_needed(DATA_ZIP_PATH, EXTRACT_DIR)

    # ------------------------------------------------------------------
    # The ZIP has top-level folder 'inaturalist_12K' inside it.
    # After extraction, we get:
    #   ../inaturalist_data/inaturalist_12K_extracted/
    #       inaturalist_12K/
    #           train/
    #           val/
    #           ...
    #
    # So the image path must include "inaturalist_12K".
    # ------------------------------------------------------------------

    # 2) Pick a single image path from the extracted data
    #    For example, one file from 'train/Plantae' 
    sample_image_path = os.path.join(
        EXTRACT_DIR,
        "inaturalist_12K",    # top-level folder from the zip
        "train",
        "Insecta",
        "0a4a6a25d2b409ed0755097ed21fdf5b.jpg"
    )
    if not os.path.isfile(sample_image_path):
        raise FileNotFoundError(f"Sample image not found at {sample_image_path}")

    # 3) Load and transform the image
    image_tensor = load_single_image(sample_image_path, resize=True, resize_dim=(32*15, 32*15))

    # Inspect the shape to pick your image_height, image_width
    # For example, if the printed shape is [3, 480, 640], do:
    _, c, h, w = image_tensor.shape


    # 4) Create model instance
    #    Example: 16 filters each conv, kernel_size=3, dense of 128
    model = MyCNN(in_channels=3,
                  num_filters=16,
                  kernel_size=3,
                  activation_fn=nn.ReLU,
                  dense_neurons=128,
                  image_height=h,
                  image_width=w)
    print(f"Model created with input shape: (3, {h}, {w})")

    # 5) Test forward pass
    test_model_with_image(model, image_tensor)

if __name__ == "__main__":
    main()


Directory ../inaturalist_data/nature_12K_extracted already exists. Skipping extraction.
Resizing image to resize_dim[0] x resize_dim[1] ...
Original image size: (800, 534) (Width x Height)
Transformed image shape: torch.Size([3, 480, 480]) (C, H, W)
Final input shape to the model: torch.Size([1, 3, 480, 480]) (Batch, C, H, W)

Model output shape: torch.Size([1, 10]) (Batch, 10)
Raw output logits: tensor([[-0.0470,  0.0263, -0.0161,  0.0578,  0.0626, -0.0428, -0.0777, -0.0319,
         -0.0444,  0.0573]])


# Q2 - Solution

In [35]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
from torch.utils.data import random_split, DataLoader
import wandb
from PIL import Image
import math
import yaml

# -------------------------------------------------------------------------
# 1) Our CNN model from Q1, extended with optional BN, dropout, filter org
# -------------------------------------------------------------------------
class MyCNN(nn.Module):
    def __init__(self,
                 in_channels=3,
                 num_filters=16,       # base number of filters
                 kernel_size=3,
                 activation_fn=nn.ReLU,
                 dense_neurons=128,
                 image_height=224,
                 image_width=224,
                 filter_organization="same",
                 batch_norm=False,
                 dropout_rate=0.0,
                 ):
        """
        :param filter_organization: "same", "double_each_layer", "halve_each_layer" etc.
        :param batch_norm: if True, add nn.BatchNorm2d after each conv
        :param dropout_rate: if > 0, we add nn.Dropout(...) in the final dense layers
        """
        super(MyCNN, self).__init__()

        Act = activation_fn  # convenience
        padding = kernel_size // 2

        # Decide how many filters each conv layer has
        # Example logic:  "same" => all layers have the same # (num_filters).
        #                 "double_each_layer" => [m, 2m, 4m, 8m, 16m]
        #                 "halve_each_layer" => [m, m/2, m/4, m/8, m/16] etc
        filter_sizes = []
        if filter_organization == "same":
            filter_sizes = [num_filters]*5
        elif filter_organization == "double_each_layer":
            filter_sizes = [num_filters*(2**i) for i in range(5)]
        elif filter_organization == "halve_each_layer":
            # integer cast for safety
            filter_sizes = [max(1, num_filters//(2**i)) for i in range(5)]
        else:
            # fallback: same
            filter_sizes = [num_filters]*5

        # We'll store each conv block in a list
        conv_layers = []
        in_ch = in_channels
        for out_ch in filter_sizes:
            block = []
            block.append(nn.Conv2d(in_ch, out_ch, kernel_size=kernel_size, stride=1, padding=padding))
            if batch_norm:
                block.append(nn.BatchNorm2d(out_ch))
            block.append(Act())
            block.append(nn.MaxPool2d(kernel_size=2, stride=2))
            conv_layers.append(nn.Sequential(*block))
            in_ch = out_ch

        self.conv_layers = nn.Sequential(*conv_layers)

        # After 5 max pools => image height/width are /32
        reduced_height = image_height // 32
        reduced_width  = image_width // 32
        final_ch = filter_sizes[-1]
        self.flatten_dim = final_ch * reduced_height * reduced_width

        # Fully connected layers
        layers_dense = []
        layers_dense.append(nn.Linear(self.flatten_dim, dense_neurons))
        if dropout_rate > 0.0:
            layers_dense.append(nn.Dropout(dropout_rate))
        layers_dense.append(Act())
        layers_dense.append(nn.Linear(dense_neurons, 10))  # 10 output classes

        self.fc = nn.Sequential(*layers_dense)

    def forward(self, x):
        # Pass through conv blocks
        x = self.conv_layers(x)
        # Flatten
        x = x.view(x.size(0), -1)
        # Dense
        x = self.fc(x)
        return x

# --------------------------------------------------------
# 2) Utility: Set random seeds for reproducibility
# --------------------------------------------------------
def set_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# --------------------------------------------------------
# 3) Create a train/val split from iNaturalist
# --------------------------------------------------------
import random
import torch
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torch.utils.data import random_split

def load_inaturalist_train_val(data_dir,
                               val_ratio=0.2,
                               seed=42,
                               augment=False,
                               resize_dim=224):
    """
    Loads the iNaturalist training data from 'data_dir' (where each subfolder 
    is a class), then splits it into train and val subsets with stratification.

    :param data_dir:      Path to the folder containing subfolders of images,
                          e.g. ".../inaturalist_12K_extracted/inaturalist_12K/train"
    :param val_ratio:     Fraction of data to reserve for validation (default 0.2)
    :param seed:          Random seed to ensure reproducible splits
    :param augment:       If True, apply creative data augmentations
    :param resize_dim:    The final resize dimension for height & width
                          (ideally a multiple of 32 for this CNN)
    :return:              (train_dataset, val_dataset, class_names)
    """

    # ----------------------------------------------------
    # 1) Define Transformations
    # ----------------------------------------------------
    # If you set augment=True, we'll apply some "creative" transformations.
    # Otherwise, we just do a simple resize + ToTensor().
    if augment:
        transform_list = [
            #  (A) Random resizing and cropping
            T.RandomResizedCrop(size=resize_dim),
            
            #  (B) Random flips
            T.RandomHorizontalFlip(p=0.5),
            
            #  (C) Some random rotation
            T.RandomRotation(degrees=30),
            
            #  (D) Color jitter (brightness/contrast/saturation/hue)
            T.ColorJitter(
                brightness=0.2,
                contrast=0.2,
                saturation=0.2,
                hue=0.1
            ),
            
            #  (E) Small chance to invert the colors
            T.RandomInvert(p=0.1),
            
            #  (F) Random perspective distortion
            T.RandomPerspective(distortion_scale=0.2, p=0.5),
            
            #  (G) Finally, convert to Tensor
            T.ToTensor(),
            
            #  (H) Optionally, random erase part of the image
            T.RandomErasing(p=0.1)
        ]
    else:
        # Minimal transform
        transform_list = [
            T.Resize((resize_dim, resize_dim)),
            T.ToTensor()
        ]

    transform = T.Compose(transform_list)

    # ----------------------------------------------------
    # 2) Create the full dataset
    # ----------------------------------------------------
    full_dataset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform)
    class_names = full_dataset.classes  # e.g. ['Amphibia', 'Animalia', ...]

    # ----------------------------------------------------
    # 3) Stratified Split
    # ----------------------------------------------------
    # We'll split the dataset so each class is represented proportionally 
    # in train and val. We do this manually because ImageFolder doesn't 
    # have direct stratified splitting out of the box.

    # Gather indices by class
    num_classes = len(class_names)
    class_indices = [[] for _ in range(num_classes)]
    
    # Fix the random seed for reproducibility
    set_seeds(seed)
    
    for idx, (_, label) in enumerate(full_dataset):
        class_indices[label].append(idx)

    train_indices = []
    val_indices   = []

    for c in range(num_classes):
        indices_c = class_indices[c]
        n_c = len(indices_c)
        n_val = int(val_ratio * n_c)
        # Shuffle indices for this class
        random.shuffle(indices_c)
        val_indices_c = indices_c[:n_val]
        train_indices_c = indices_c[n_val:]
        val_indices.extend(val_indices_c)
        train_indices.extend(train_indices_c)

    # Build subset datasets
    train_dataset = torch.utils.data.Subset(full_dataset, train_indices)
    val_dataset   = torch.utils.data.Subset(full_dataset, val_indices)

    return train_dataset, val_dataset, class_names


# --------------------------------------------------------
# 4) Training loop
# --------------------------------------------------------
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def validate_one_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# --------------------------------------------------------
# 5) Main train function that W&B calls per sweep run
# --------------------------------------------------------
def train_and_evaluate(config):
    # Initialize a W&B run
    wandb.init(project="inat_sweep_demo", config={
        "epochs": 5
    })
    sweep_config = wandb.config  # short alias

    device = "cuda" if torch.cuda.is_available() else "cpu"
    set_seeds(42)

    # Build the model
    if sweep_config.activation == "relu":
        act_fn = nn.ReLU
    elif sweep_config.activation == "gelu":
        act_fn = nn.GELU
    elif sweep_config.activation == "silu":
        act_fn = nn.SiLU
    elif sweep_config.activation == "mish":
        act_fn = nn.Mish
    else:
        act_fn = nn.ReLU  # fallback

    model = MyCNN(
        num_filters=sweep_config.num_filters,
        kernel_size=3,
        activation_fn=act_fn,
        dense_neurons=128,
        image_height=config['resize_dim'],  # or whatever your images are
        image_width=config['resize_dim'],
        filter_organization=sweep_config.filter_organization,
        batch_norm=sweep_config.batch_norm,
        dropout_rate=sweep_config.dropout_rate
    ).to(device)

    # Load data
    train_dir = os.path.join(config['data_root'], "train")
    # We'll do a random 80/20 split, but keep it consistent for the sweep
    train_dataset, val_dataset, class_names = load_inaturalist_train_val(
        data_dir=train_dir,
        val_ratio=0.2,
        seed=42,
        augment=sweep_config.data_augmentation,
        resize_dim=config['resize_dim']
    )

    train_loader = DataLoader(train_dataset, batch_size=sweep_config.batch_size, shuffle=True, num_workers=4)
    val_loader   = DataLoader(val_dataset,   batch_size=sweep_config.batch_size, shuffle=False, num_workers=4)

    # Optimizer & Loss
    optimizer = optim.Adam(model.parameters(), lr=sweep_config.learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Train loop
    for epoch in range(sweep_config.epochs):
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc = validate_one_epoch(model, val_loader, criterion, device)

        # Log to W&B
        wandb.log({
            "epoch": epoch+1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_accuracy": val_acc
        })

        print(f"Epoch {epoch+1}/{sweep_config.epochs} - "
              f"Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.3f} | "
              f"Val Loss: {val_loss:.3f}, Val Acc: {val_acc:.3f}")

    # Mark run finished
    wandb.finish()

# --------------------------------------------------------
# 6) Entry point if launching script manually
#    For sweeps, W&B will call train_and_evaluate() via agent
# --------------------------------------------------------
if __name__ == "__main__":
    # Example of how to do a single run without sweeps, for debugging:
    config = None
    with open("../config/config.yaml", "r") as file:
        config = yaml.safe_load(file)

    # For a single run, we can do:
    wandb.init(project="inat_sweep_demo")
    wandb.config.update({
        "num_filters": 32,
        "activation": "relu",
        "filter_organization": "same",
        "data_augmentation": True,
        "batch_norm": True,
        "dropout_rate": 0.2,
        "learning_rate": 1e-3,
        "batch_size": 32,
        "epochs": 2,
        "data_root": config['q2_configs']['data_root']
    })
    
    config = None
    with open("../config/config.yaml", "r") as file:
        config = yaml.safe_load(file)
    
    train_and_evaluate(config['q2_configs'])

# Usage: python train_sweep.py --data_root ../inaturalist_data/nature_12K_extracted/inaturalist_12K


AttributeError: <class 'wandb.sdk.wandb_config.Config'> object has no attribute 'activation'